In [7]:
from __future__ import division, print_function
8888888
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Dropout, Reshape, Flatten, concatenate, Input, Embedding
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import os
import collections
import re
import string

### Read data

In [10]:
import pandas as pd

#Data Input and Preprocessing
data = pd.read_csv('reddit.csv')
#test = pd.read_csv('test2_19.csv')
for row in range(len(data["id"])):
    #print(data["id"][row])
    data["id"][row]=(data["id"][row].splitlines())
    for num in range(len(data["id"][row])):
        strin=''.join(data["id"][row][num].split())
        newstr='';
        for i in range(len(strin)-2):
            newstr+=strin[i+2]
        data["id"][row][num]=newstr

for row in range(len(data["hate_speech_idx"])):
        if isinstance(data["hate_speech_idx"][row],float):
            data["hate_speech_idx"][row]="[0]" 

data_mod=[[]]
            
for row in range(len(data["text"])):
    #print(data["id"][row])
    data["text"][row]=(data["text"][row].splitlines())
    for num in range(len(data["text"][row])):
        strin=' '.join(data["text"][row][num].split())
        newstr='';
        for i in range(len(strin)-2):
            newstr+=strin[i+2]
        data["text"][row][num]=newstr
        text = False
        for i in range(len(data["hate_speech_idx"][row])):
            if (data["hate_speech_idx"][row][i]!='[' and data["hate_speech_idx"][row][i]!=' ') and (data["hate_speech_idx"][row][i]!=']' and data["hate_speech_idx"][row][i]!=','):
                if int(num)+1 == int(data["hate_speech_idx"][row][i]):
                    line=[]
                    line.append(newstr)
                    line.append(1)
                    data_mod.append(line)
                    text = True
                    break
        if text == False:
            line=[]
            line.append(newstr)
            line.append(0)
            data_mod.append(line)
            
        

data_mod.remove([])        
print(data_mod[0][0])

data = pd.DataFrame(data_mod, columns = ['Text', 'Label'])


 A subsection of retarded Hungarians? Ohh boy. brace for a livid Bulbasaur coming in here trying to hate a hole in some of her stupider countrymen.


In [11]:
data.head()

,Text,Label
0,A subsection of retarded Hungarians? Ohh boy....,1
1,Hiii. Just got off work. 444 is mainly the ty...,0
2,wow i guess soyboys are the same in every cou...,0
3,Owen Benjamin's soyboy song goes for every co...,0
4,"> ""y'all hear sumn?"" by all means I live in a...",0


In [12]:
data.Label.unique()

array([1, 0])

In [13]:
data.shape

(22324, 2)

In [14]:
pos = []
neg = []
for l in data.Label:
    if l == 0:
        pos.append(0)
        neg.append(1)
    elif l == 1:
        pos.append(1)
        neg.append(0)

In [15]:
data['Pos']= pos
data['Neg']= neg

In [16]:
data.head()

,Text,Label,Pos,Neg
0,A subsection of retarded Hungarians? Ohh boy....,1,1,0
1,Hiii. Just got off work. 444 is mainly the ty...,0,0,1
2,wow i guess soyboys are the same in every cou...,0,0,1
3,Owen Benjamin's soyboy song goes for every co...,0,0,1
4,"> ""y'all hear sumn?"" by all means I live in a...",0,0,1


### Clean data

In [17]:
def remove_punct(text):
    text_nopunct = ''
    text_nopunct = re.sub('['+string.punctuation+']', '', text)
    return text_nopunct

data['Text_Clean'] = data['Text'].apply(lambda x: remove_punct(x))
print(data["Text_Clean"].head())

0     A subsection of retarded Hungarians Ohh boy b...
1     Hiii Just got off work 444 is mainly the typa...
2     wow i guess soyboys are the same in every cou...
3     Owen Benjamins soyboy song goes for every cou...
4      yall hear sumn by all means I live in a smal...
Name: Text_Clean, dtype: object


In [34]:
from nltk import word_tokenize, WordNetLemmatizer
tokens = [word_tokenize(sen) for sen in data.Text_Clean]
print(type(tokens[0]))
print(type(tokens[0][0][0]))

<class 'list'>
<class 'str'>


In [30]:
def lower_token(tokens): 
    return [w.lower() for w in tokens]    
    
lower_tokens = [lower_token(token) for token in tokens] 
print(type(lower_tokens[0]))
print(type(lower_tokens[0][0]))

<class 'list'>
<class 'str'>


In [31]:
from nltk.corpus import stopwords
stoplist = stopwords.words('english')

In [22]:
def remove_stop_words(tokens):
    return [word for word in tokens if word not in stoplist]

In [37]:
filtered_words = [remove_stop_words(sen) for sen in lower_tokens]
print(filtered_words[0])

['subsection', 'retarded', 'hungarians', 'ohh', 'boy', 'brace', 'livid', 'bulbasaur', 'coming', 'trying', 'hate', 'hole', 'stupider', 'countrymen']


In [38]:
result = [' '.join(sen) for sen in filtered_words]

In [42]:
print(result[0])
data['Text_Final'] = result
type(data)

subsection retarded hungarians ohh boy brace livid bulbasaur coming trying hate hole stupider countrymen


pandas.core.frame.DataFrame

In [103]:
data['tokens'] = filtered_words

In [104]:
data = data[['Text_Final', 'tokens', 'Label', 'Pos', 'Neg']]

In [105]:
data[:4]

,Text_Final,tokens,Label,Pos,Neg
0,subsection retarded hungarians ohh boy brace l...,"[subsection, retarded, hungarians, ohh, boy, b...",1,1,0
1,hiii got work 444 mainly typa guys imagine wri...,"[hiii, got, work, 444, mainly, typa, guys, ima...",0,0,1
2,wow guess soyboys every country,"[wow, guess, soyboys, every, country]",0,0,1
3,owen benjamins soyboy song goes every country ...,"[owen, benjamins, soyboy, song, goes, every, c...",0,0,1


### Split data into test and train

In [106]:
data_train, data_test = train_test_split(data, test_size=0.20, random_state=42)

In [107]:
all_training_words = [word for tokens in data_train["tokens"] for word in tokens]
training_sentence_lengths = [len(tokens) for tokens in data_train["tokens"]]
TRAINING_VOCAB = sorted(list(set(all_training_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_training_words), len(TRAINING_VOCAB)))
print("Max sentence length is %s" % max(training_sentence_lengths))

424838 words total, with a vocabulary size of 33666
Max sentence length is 763


In [108]:
all_test_words = [word for tokens in data_test["tokens"] for word in tokens]
test_sentence_lengths = [len(tokens) for tokens in data_test["tokens"]]
TEST_VOCAB = sorted(list(set(all_test_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_test_words), len(TEST_VOCAB)))
print("Max sentence length is %s" % max(test_sentence_lengths))

107122 words total, with a vocabulary size of 16294
Max sentence length is 623


### Load Google News Word2Vec model

In [109]:
word2vec_path = 'GoogleNews-vectors-negative300.bin.gz'
word2vec = models.KeyedVectors.load_word2vec_format(word2vec_path, binary=True)

In [110]:
def get_average_word2vec(tokens_list, vector, generate_missing=False, k=300):
    if len(tokens_list)<1:
        return np.zeros(k)
    if generate_missing:
        vectorized = [vector[word] if word in vector else np.random.rand(k) for word in tokens_list]
    else:
        vectorized = [vector[word] if word in vector else np.zeros(k) for word in tokens_list]
    length = len(vectorized)
    summed = np.sum(vectorized, axis=0)
    averaged = np.divide(summed, length)
    return averaged

def get_word2vec_embeddings(vectors, clean_comments, generate_missing=False):
    embeddings = clean_comments['tokens'].apply(lambda x: get_average_word2vec(x, vectors, 
                                                                                generate_missing=generate_missing))
    return list(embeddings)

### Get Embeddings

In [111]:
training_embeddings = get_word2vec_embeddings(word2vec, data_train, generate_missing=True)

In [112]:
MAX_SEQUENCE_LENGTH = 50
EMBEDDING_DIM = 300

### Tokenize and Pad sequences

In [113]:
tokenizer = Tokenizer(num_words=len(TRAINING_VOCAB), lower=True, char_level=False)
tokenizer.fit_on_texts(data_train["Text_Final"].tolist())
training_sequences = tokenizer.texts_to_sequences(data_train["Text_Final"].tolist())

train_word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(train_word_index))

Found 33596 unique tokens.


In [114]:
train_cnn_data = pad_sequences(training_sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [115]:
train_embedding_weights = np.zeros((len(train_word_index)+1, EMBEDDING_DIM))
for word,index in train_word_index.items():
    train_embedding_weights[index,:] = word2vec[word] if word in word2vec else np.random.rand(EMBEDDING_DIM)
print(train_embedding_weights.shape)

(33597, 300)


In [116]:
test_sequences = tokenizer.texts_to_sequences(data_test["Text_Final"].tolist())
test_cnn_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)

### Define RNN-LSTM

In [117]:
label_names = ['Pos', 'Neg']

In [118]:
y_train = data_train[label_names].values

In [119]:
x_train = train_cnn_data
y_tr = y_train

In [120]:
def recurrent_nn(embeddings, max_sequence_length, num_words, embedding_dim, labels_index):
    
    embedding_layer = Embedding(num_words,
                            embedding_dim,
                            weights=[embeddings],
                            input_length=max_sequence_length,
                            trainable=False)
    
    sequence_input = Input(shape=(max_sequence_length,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)

#     lstm = LSTM(256, dropout=0.2, recurrent_dropout=0.2, return_sequences=True)(embedded_sequences)
    lstm = LSTM(256)(embedded_sequences)
    
    x = Dense(128, activation='relu')(lstm)
    x = Dropout(0.2)(x)
    preds = Dense(labels_index, activation='sigmoid')(x)

    model = Model(sequence_input, preds)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['acc'])
    model.summary()
    return model

In [121]:
model = recurrent_nn(train_embedding_weights, MAX_SEQUENCE_LENGTH, len(train_word_index)+1, EMBEDDING_DIM, 
                len(list(label_names)))

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 50)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 50, 300)           10079100  
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               570368    
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 258       
Total params: 10,682,622
Trainable params: 603,522
Non-trainable params: 10,079,100
_________________________________________

### Train RNN-LSTM

In [122]:
num_epochs = 5
batch_size = 34

In [123]:
hist = model.fit(x_train, y_tr, epochs=num_epochs, validation_split=0.1, shuffle=True, batch_size=batch_size)

Train on 16073 samples, validate on 1786 samples
Epoch 1/5
16073/16073 [==============================] - 70s 4ms/step - loss: 0.4107 - acc: 0.8376 - val_loss: 0.3579 - val_acc: 0.8791
Epoch 2/5
16073/16073 [==============================] - 66s 4ms/step - loss: 0.3457 - acc: 0.8767 - val_loss: 0.3448 - val_acc: 0.8858
Epoch 3/5
16073/16073 [==============================] - 65s 4ms/step - loss: 0.3357 - acc: 0.8800 - val_loss: 0.3410 - val_acc: 0.8841
Epoch 4/5
16073/16073 [==============================] - 65s 4ms/step - loss: 0.3284 - acc: 0.8821 - val_loss: 0.3379 - val_acc: 0.8838
Epoch 5/5
16048/16073 [============================>.] - ETA: 0s - loss: 0.3141 - acc: 0.8848

/home/kanishk/miniconda3/lib/python3.7/site-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_test_batch_begin) is slow compared to the batch update (0.191753). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


16073/16073 [==============================] - 71s 4ms/step - loss: 0.3139 - acc: 0.8849 - val_loss: 0.3361 - val_acc: 0.8835


### Test RNN-LSTM

In [124]:
predictions = model.predict(test_cnn_data, batch_size=1024, verbose=1)

4465/4465 [==============================] - 7s 2ms/step


In [125]:
labels = [1, 0]

In [135]:
prediction_labels=[]
for p in predictions:
    prediction_labels.append(labels[np.argmax(p)])

#### Computing the Confusion Matrix ####
from sklearn.metrics import confusion_matrix, accuracy_score, fbeta_score
y_pred_class = prediction_labels
y_true = data_test.Label
tn, fp, fn, tp = confusion_matrix(y_true, y_pred_class).ravel()
accuracy = (tp + tn) / (tp + fp + fn + tn)
print("Accuracy="+str(accuracy*100)+"%")

Accuracy=87.77155655095184%


In [137]:
beta = 1  ## For F1 score
print("F1 SCORE = "+str(fbeta_score(y_true, y_pred_class, beta)*100))

F1 SCORE = 73.64864864864865


In [138]:
from sklearn.metrics import roc_auc_score
roc_auc = roc_auc_score(y_true, y_pred_pos)
print("ROC_AUC SCORE = "+str(roc_auc_score(y_true, y_pred_pos)*100))

NameError: name 'y_pred_pos' is not defined